In [ ]:
import argparse
import gym
from gym import wrappers
import os.path as osp
import random
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers
import os
os.chdir("atari")
from run_dqn_atari import *
import dqn
from dqn_utils import *
from atari_wrappers import *
import pickle

In [14]:
# Get Atari games.
benchmark = gym.benchmark_spec('Atari40M')
# Change the index to select a different game.
task = benchmark.tasks[3]
# Run training
seed = 0 # Use a seed of zero (you may want to randomize the seed!)
env = get_env(task, seed)
sess = get_session()
num_timesteps=task.max_timesteps
   
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('pong.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))
#q_func=atari_model

INFO:tensorflow:Restoring parameters from ./pong


In [7]:
q_func_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='q_func')
target_q_func_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_q_func')

In [ ]:
replay_buffer_size=2
frame_history_len=4
replay_buffer = ReplayBuffer(replay_buffer_size, frame_history_len)

In [ ]:
graph = tf.get_default_graph()
Argmax= graph.get_tensor_by_name("ArgMax:0")
img_Input=graph.get_tensor_by_name("Placeholder:0")
img_Input1=graph.get_tensor_by_name("Placeholder_1:0")
img_Input2=graph.get_tensor_by_name("Placeholder_2:0")
img_Input3=graph.get_tensor_by_name("Placeholder_3:0")
img_Input4=graph.get_tensor_by_name("Placeholder_4:0")

In [ ]:
model_initialized = False
num_param_updates = 0
mean_episode_reward      = -float('nan')
best_mean_episode_reward = -float('inf')
last_obs = env.reset()
LOG_EVERY_N_STEPS = 10000

In [ ]:
for t in itertools.count():
    if stopping_criterion is not None and stopping_criterion(env, t):
        break
    idx = replay_buffer.store_frame(last_obs)   
    if t == 0:
        act, reward, done = env.action_space.sample(), 0, False
        replay_buffer.store_effect(idx, act, reward, done)
    last_obs, reward, done, info = env.step(act)
    idx = replay_buffer.store_frame(last_obs) 
    replay_buffer.store_effect(idx, act, reward, done)
    
    obs_batch      = replay_buffer._encode_observation(0)[None]
    act_batch      = replay_buffer.action[[0]]
    rew_batch      = replay_buffer.reward[[0]]
    next_obs_batch = replay_buffer._encode_observation(1)[None]
    done_mask      = np.array([0.0])
    
    input_batch = replay_buffer._encode_sample(0)
#     q_vals = session.run(q_func_1, {obs_t_ph: input_batch[None, :]})
#     act = np.argmax(q_vals)
#     obs_t_float   = tf.cast(input_batch[None, :],   tf.float32) / 255.0
#     q_vals=q_func(obs_t_float, num_actions, scope="q_func", reuse=False)
    
    act=sess.run(Argmax,{img_Input:obs_batch,img_Input1:act_batch,img_Input2:rew_batch,img_Input3:next_obs_batch,img_Input4:done_mask})
    
    
    
    episode_rewards = get_wrapper_by_name(env, "Monitor").get_episode_rewards()
    if len(episode_rewards) > 0:
            rewards_record.append(episode_rewards[-1])

    if len(episode_rewards) > 0:
            mean_episode_reward = np.mean(episode_rewards[-100:])
            mean_episode_reward_record.append(mean_episode_reward)

    if len(episode_rewards) > 100:
        best_mean_episode_reward = max(best_mean_episode_reward, mean_episode_reward)
        best_mean_episode_reward_record.append(best_mean_episode_reward)
    
    if t % LOG_EVERY_N_STEPS == 0 and model_initialized:
        print("Timestep %d" % (t,))
        print("mean reward (100 episodes) %f" % mean_episode_reward)
        print("best mean reward %f" % best_mean_episode_reward)
        print("episodes %d" % len(episode_rewards))
        print("exploration %f" % exploration.value(t))
        print("learning_rate %f" % optimizer_spec.lr_schedule.value(t))
        sys.stdout.flush()
        record = ({"rewards":rewards_record, "mean_rewards": mean_episode_reward_record, "best_mean": best_mean_episode_reward_record})
        with open("./rewards.pickle", "wb") as f:
            pickle.dump(record, f)
    